In [17]:
#预测
grid_xgb.predict(Xtest)

array([0, 0, 0, ..., 0, 0, 1])

In [16]:
grid_xgb.best_params_

{'colsample_bytree': 0.8, 'n_estimators': 100, 'subsample': 1}

In [15]:
# 网格搜索调参
from sklearn.model_selection import GridSearchCV
# 参数字典 np.linspace(100,160,4),会产生小数
param = {'n_estimators':np.arange(100,160,30),
         'subsample':[0.8,0.9,1],
         'colsample_bytree':[0.8,0.9,1]}
xgb_2 = XGBClassifier()
# 以下cv是3折交叉验证
grid_xgb = GridSearchCV(xgb_2,param,cv=3,n_jobs=-1)
grid_xgb.fit(Xtrain,Ytrain)
print(classification_report(Ytest, grid_xgb.predict(Xtest)))

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 0.9, 1],
                         'n_estimators': array([100, 130]),
                         'subsample': [0.8, 0.9, 1]})

              precision    recall  f1-score   support

           0       0.68      0.80      0.74      7832
           1       0.60      0.44      0.50      5247

    accuracy                           0.66     13079
   macro avg       0.64      0.62      0.62     13079
weighted avg       0.65      0.66      0.64     13079



In [8]:
# 推荐使用更加友好的sklearn接口
# 训练模型Baseline
#不用原生接口，并且 Xgbbost能够实现分类回归两种任务
from xgboost.sklearn import XGBClassifier
# 推荐可调
# n_estimators:树的最大数量，默认100，建议50-100
# max_depth: 默认3，推荐3-10
# max_leaves：叶结点数量，不用调
# learning_rate=0.1,学习率，建议0.001~0.1，控制递度下降的步长（反复尝试迭代，先找斜率最大），根据上山找下山的路线
# reg_alpha = 0 L1正则 ，用于特征选择，Lasso回归（陡峭），可用于变量选择，太陡的不要
# reg_lambda = 1 L2正则，用于过拟合,岭回归（线路平缓）
# 下面需要设置
# objective = 'binary:logistic'二分；‘multi:softmax'多分类 ,
# booster = gbtree默认GBDT,gblinear逻辑回归 or dart带剪枝的树（不推荐，因为已经有正则）

# subsample = 1，0.5~1，不抽样 样本随机采样，无放加抽样（随机森林是有放回），0.8抽80%
# colsample_bytree = 1 0.5~1，特征随机采样，无放回，0.8抽80%的列
# tree_method: 对应 lightGBM，(auto默认；hist,gpu_hist显卡配置并支持深度计算才能用)
xgb = XGBClassifier()
xgb.fit(Xtrain,Ytrain)
#.这个没有score
# 模型评估
from sklearn.metrics import classification_report
print(classification_report(Ytest, xgb.predict(Xtest)))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

              precision    recall  f1-score   support

           0       0.68      0.81      0.74      7832
           1       0.61      0.44      0.51      5247

    accuracy                           0.66     13079
   macro avg       0.65      0.63      0.63     13079
weighted avg       0.66      0.66      0.65     13079



In [4]:
#原理，GBDT的加法模型，把输出的值，全部相加就行，G：对应的是残差； B； 指boosting
#残差，指模型没有办法解释的信息
#XGBoost,陈天奇，2014年博士论文，引入了泰勒展开式和正则项（解决过拟合）
 # 。极限递度提升树
#泰勒二阶展开式，成立条件，损失函数必需可导，层数的结果一定是最优
df = pd.read_csv('保险数据_全部数据.csv')
df1 = df.dropna()
#分割标签
labels = df1.pop('resp_flag')
# 数据编码,label这个库可以做数据编码
from sklearn.preprocessing import LabelEncoder
for i in df1.columns:
    if df1[i].dtype =='object':
        df1[i] = LabelEncoder().fit_transform(df1[i])
from sklearn.model_selection import train_test_split

Xtrain, Xtest, Ytrain, Ytest = train_test_split(df1, labels,
                                                test_size=0.3,
                                                random_state=42,
                                                stratify=labels)


In [ ]:
#GBDT, 递度提升树，全部是回归树(不是分类树）
#，真正的递度数据，进化为残差上的操作
#，GBDT三个概念：1.Regressinon Decision Tree 回归树。残差平方和最小
 # 2.Gradient Boosting(GB) ,残差树，一次一次的构建不同的残差树
 # 3.Shrinkage(缩减），每次一小步，正逐渐逼近结果，不完全信任每一棵残差树step调整
    #,小步快跑，避免过度拟合
#决策树进化史，ID3 -> C4.5 -> cart
 #ID3: 只能解决分类问题，不能解决回归问题
 #C4.5: 回归树不做超出训练集的数据预测，因为两侧都是一条直线

In [2]:
import os
os.chdir('C:\Data\Jupyter_file\机器学习')

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")